# Project 1
An introduction to locally weighted regression.



## What is Regression?

Regression is a technique of predicting an output, or dependent variable, given one or more inputs, or independent variables.

Mathematically, let $X$ be the inputs and $y$ be the outputs. Then, the regression equation can be shown as $$\mathbb{E}(y|X) = f(X).$$

Taking the expectation of the outputs demonstrates a minimization of the **mean squared error (MSE)**: the squared difference between the data points and the estimator function. By minimizing this, we can reasonably be sure that we have a good estimator function, based on the data we have.

**Linear regression** specifies the use of a straight line, rather than a curve. There are also usually weights for each feature in $X$, which put more emphasis on certain features based on which features are deemed more "important", or seem to affect the outputs more.

So, let $X={x_1,x_2,...,x_n}$ be the inputs, $W={w_1,w_2,...,w_n}$ be the weights, and $y$ be the outputs.  Then the linear regression model is $$y=X \cdot W=x_1 \cdot w_1 + x_2 \cdot w_2 + ... + x_n \cdot w_n$$

## Locally Weighted Regression

Locally weighted regression is a method of executing linear regression on data with nonlinear trends and/or associations.  However, rather than using the whole data set for the regression, locally weighted regression evaluates the regression at multiple points of interest, with the weights based on the data points' distance from the point of interest.

For example, in the image below, the point of interest is the vertical line, and since the points nearest the point of interest have a negative linear trend, the line of regression has a negative slope.  The points farther away from the point of interest demonstrate a nonlinear trend, but since they are farther away, their weights are smaller (if not zero) and have little to no influence on the line of regression.

![img154.gif](https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume4/cohn96a-html/img154.gif)

Locally weighted regression typically relies on a specified kernel to calculate the weights.  Some common ones are Gaussian, Exponential, Tricubic, Epanechnikov, and Quartic.  These kernels determine the shape of the curve that calculates the weights, placing more or less significance on the surrounding points depending on the kernel.